# Differential cross section

In [11]:
%matplotlib inline

In [3]:
# %load "../../style.py"
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import colorConverter
from matplotlib import rcParams
import seaborn as sns
from scipy.optimize import curve_fit

sns.set(style='ticks', palette='Set2') 
sns.despine()

# These are the colors. Notice how this is programmed:
# You initialize your colors by 
# colorset = palette()
# then you can cycle through the colors:
# color = next(colorset)
# if you want your set to be reset, just create
# a new palette() instance! This way the colors do not interfere.

color_names = ['windows blue', "pale red", "faded green", "amber", 
          'dark green', 'dark fuchsia', 'browny orange', 
          'puke green', 'dark royal blue', 'dusty purple', 'red orange']
colors = sns.xkcd_palette(color_names)
palette = lambda: itertools.cycle(sns.xkcd_palette(color_names) )

fontsize_labels = 26    # size used in latex document
rcParams['text.latex.preamble'] = [r'\usepackage[cmbright]{sfmath}']
rcParams['font.family']= 'sans-serif'
rcParams['font.sans-serif']= 'cmbright'
rcParams['font.weight'] = "light"

rcParams['text.usetex'] = True

rcParams['figure.autolayout'] = True
rcParams['font.size'] = fontsize_labels
rcParams['axes.labelsize'] = fontsize_labels
rcParams['xtick.labelsize'] = fontsize_labels
rcParams['ytick.labelsize'] = fontsize_labels
rcParams['legend.fontsize'] = fontsize_labels
rcParams['legend.markerscale'] = 4
rcParams['axes.titlesize'] = fontsize_labels
rcParams['text.color'] = "0.3"
rcParams['xtick.color'] = "0.3"
rcParams['ytick.color'] = "0.3"
rcParams['axes.labelcolor'] = "0.3"
rcParams['axes.edgecolor'] = "0.8"

xfactor = 2
rcParams['figure.figsize'] = (xfactor*6.2, xfactor*3.83)  

save_fig = True
if not save_fig:
        rcParams['figure.figsize'] = (13, 8) 
fig_dir = "./figures/"  # directory of figures

def fixticks(ax):    
    for t in ax.xaxis.get_ticklines(): t.set_color('0.8')
    for t in ax.yaxis.get_ticklines(): t.set_color('0.8')


In [5]:
# %load ../../tools.py
def uc_str(c, max_digit=4):
    """
    input format: uc.ufloat
    rounds float and corrisponding error to last significant digit
    returns float and error as string
    as integers with max max_digit (=4) error digits
    as floats with max 4 error digits
    as exp else
    """
    digit = -int(np.floor(np.log10(c.s)))    
    if (c.s * 10**digit) < 1.5: # convention...
        digit += 1
    c_r = round(c.n, digit)
    s_c_r = round(c.s, digit)
    if (-3 < digit) * (digit <= 0): # returns readable integers
        c_str = '%i \pm %i'%(c_r, s_c_r)
    elif (0 < digit) * (digit < (max_digit + 1)): # returns readable floats (max 3 digits)
        c_str = ('%.' + str(digit) + 'f \pm %.' + str(digit) + 'f')%(c_r, s_c_r)
    else: # returns exp
        c_str = ('(%.1f \pm %.1f)\mathrm{e}%i')%(c_r * 10**(digit-1), s_c_r * 10**(digit-1), -(digit-1))
    return c_str

def enum(arr1, *args):
    i_range = range(len(arr1))
    return zip(i_range, arr1 ,*args)


In [7]:
# %load ../../preamb.py
import numpy as np
import uncertainties as uc
import uncertainties.unumpy as un
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter as sav
from scipy.special import erfc
from scipy.integrate import quad
import sys
import re

npy_dir = "./npy/"


## Integration

Photon intensities at different angles. 
Integration is done numerically: sum over entries of histogram, 
multiplied by width of bin in energy domain (result of calibration_na). 
The differential cross section is approximated by the fraction of 
photons scattered at angle $\theta$ ($I(\theta)$) over the total incident 
intensity ($I_{tot}$), divided by the solid angle $\Delta \Omega$.

In [14]:
dE = na_cal[0] * 16
i_photon = dE * np.load(npy_dir + 'i_photon_' + str(theta) + '.npy')

"""
# Calculate integral directly
sums = []
theta = 75
y = np.load(npy_dir + 'na_rate_' + str(theta) + '.npy')
sums.append(np.sum(y))
na_cal = np.load(npy_dir + 'na_calibration.npy')
integral = np.array(sums) * dE
"""

# no_total_incident

'epsilon(energy)'
'mu(energy)'

NameError: name 'na_cal' is not defined

In [ ]:
def plot_total():
    '''
    Plots total incident photons at NaI scintillator
    (source: 137Cs, no target in between)
    '''
    #### GET DATA  ####
    file_name = "na_total_incident"
    file_in = npy_dir + file_name + '.npy'
    y = np.load(file_in)
    y_e = un.uarray(y, np.maximum(1, np.sqrt(y)))
    
    ### Get time from file
    file_in_mcd = data_dir + file_name + '.TKA'
    f = open(file_in_mcd)
    settings = f.read()
    f.close()
    lines = settings.split('\n')
    t = np.float(lines[1]) # livetime is written in line 1...

    rate = y / t
    #rate = rate - rate_bg
    #rate = rate - rate_rnd
    rate[rate < 0] = 0
    y = rate    # Continue to work with the rate!

    rate_e = y_e / t # - rate_bg_e - rate_rnd_e
    rate_e[rate < 0] = 0 


    # Rebinning: 1/16 of number of bins
    z = y[:-14] # last 14 bins are dropped to obtain a len(z) as a multiple of 16
    z = z.reshape([len(z) / 16, 16])
    z = np.sum(z, axis=1)
    y = z

    z_e = rate_e[:-14] # last 14 bins are dropped to obtain a len(z) as a multiple of 16
    z_e = z_e.reshape([len(z_e) / 16, 16])
    z_e = np.sum(z_e, axis=1)
    y_e = z_e
    np.save(npy_dir + 'na_rate_' + str(theta), y_e)

    x = np.arange(len(y))
    y_filtered = sav(y, 201, 7)

    ###### PLOTTING #################
    if show_fig:
        fig1, ax1 = plt.subplots(1, 1)
        if not save_fig:
            fig1.suptitle("NaI scintillator, total incident photons")
        ax1.plot(x, y, '.', alpha=0.9, label='data')
        ax1.set_xlabel("Channel")
        ax1.set_ylabel("Rate / $s^{-1}$")
        ax1.set_xlim(0, 800)
        ax1.set_ylim(0,)
        ax1.legend(loc=1)
        ax1.grid(True)
        if save_fig:
            fig1.savefig(fig_dir + file_name + ".pdf")
            fig1.savefig(fig_dir + file_name + ".png")

    return(0)


In [ ]:
plot_total()

must further include: errors on counts for all measurements